In [ ]:
#----------------------------------------------------------------------------------
# Author:        Damien Robinson
# Date:          16/07/2018
# Description:   A Jupyter Notebook to security harden ArcGIS Enterprise
#                
#
# (C) Copyright ESRI (UK) Limited 2018. All rights reserved
# ESRI (UK) Ltd, Millennium House, 65 Walton Street, Aylesbury, HP21 7QG
# Tel: +44 (0) 1296 745500  Fax: +44 (0) 1296 745544
#----------------------------------------------------------------------------------

#Import Statements
import logging
import json
from arcgis.gis import GIS
from arcgis.gis import server

#Variables (To be updated for each deployment)
url = "https://yourProject.cloud.esriuk.com/portal"
username = "yourUsername"
password = "yourPassword"
logName = "yourLogName.log"

#Derived Variables
baseURL = url[:-6]
serverURL = baseURL+"server"

#Logging
# create logger
logger = logging.getLogger('ArcGISEnterprise_Hardening')
logger.setLevel(logging.DEBUG)
# create log file
fh = logging.FileHandler(logName)
fh.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(logging.DEBUG)
#Set Log Format
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# Add log file and console handlers
logger.addHandler(fh)
logger.addHandler(ch)



logger.info('Hardening security on the following ArcGIS Enterprise - '+url)


#Functions
logger.info('Defining configureGeneralProperties function')
def configureGeneralProperties(gis):
    logger.info("Configuring general properties")

    try:
        if generalProperties == None or len(generalProperties) == 0:
            logger.info("No general properties to apply")
            return True

        updProperties = generalProperties
        result = gis.update_properties(updProperties)
        return result
    except Exception as ex:
        logger.error("ERROR: Unexpected exception thrown", ex)
        return False
    
logger.info('Defining configureSecurity function')
def configureSecurity(gis):
    logger.info("Configuration security options")

    try:
        if securityConfig == None or len(securityConfig) == 0:
            logger.info("No security configuration to apply")
            return True

        updConfig = json.loads(json.dumps(gis.admin.security.config))
        #
        # Remove items we are not expecting
        #
        for k in ["userStoreConfig", "groupStoreConfig"]:
            if k in updConfig:
                del updConfig[k]


        #
        # Add in our changes
        #
        for k in securityConfig:
            updConfig[k] = securityConfig[k]
        #
        # If the defaultRoleForUser property is not account_user or account_publisher, search the
        # roles to find the named role and substitute it for the role id
        #
        if "defaultRoleForUser" in securityConfig and \
            securityConfig["defaultRoleForUser"] not in ["account_user", "account_publisher"]:

            roleName = securityConfig["defaultRoleForUser"]
            existingRole = _findRole(gis, roleName)
            if existingRole == None:
                logger.error("ERROR: Role defined for defaultRoleForUser property does not exist: " + roleName)
                return False
            updConfig["defaultRoleForUser"] = existingRole.role_id
        #
        # Update the configuration
        #
        gis.admin.security.config = updConfig
        return True
    except Exception as ex:
        logger.error("ERROR: Unexpected exception thrown", ex)
        return False

logger.info('Defining configureSystemProperties function')
def configureSystemProperties(gis):
    logger.info("Configuring system properties")

    try:
        if systemProperties == None or len(systemProperties) == 0:
            logger.info("No system properties to apply")
            return True

        system = gis.admin.system
        curProperties = system.properties
        updProperties = json.loads(json.dumps(curProperties))
        for k in systemProperties:
            if k == "disableSignup":
               if systemProperties[k] == True:
                systemProperties[k] = "true"
               elif systemProperties[k] == False:
                systemProperties[k] = "false"
            updProperties[k] = systemProperties[k]
        #
        # Check that we are actually going to update something, as calling update
        # will cause Portal to be restarted.
        #
        haveChanges = False
        for k in curProperties:
            if updProperties[k] != curProperties[k]:
                haveChanges = True
        for k in updProperties:
            if k not in curProperties:
                haveChanges = True

        if not haveChanges:
            logger.info("No system properties require changing")
            return True
        else:
            #
            # Using the following sends a gis.admin.system._con.get() call, which does not seem to work.
            # gis.admin.system.properties = updProperties
            # The following, therefore, uses a post command
            url = system._url + "/properties/update"
            params = {
                "f" : "json",
                "properties" : updProperties
            }
            logger.info("  Updating system properties - this may take several minutes")
            result = system._con.post(path=url, postdata=params)
            if "status" in result:
                return result["status"] == "success"
            else:
                return result
    except Exception as ex:
        logger.error("ERROR: Unexpected exception thrown", ex)
        return False

def disableAGSServicesDirectory(ags):
    logger.info("Disabling ArcGIS Server Services Directory")

    try:
        if servicesDirectory == None or len(servicesDirectory) == 0:
            logger.info("No services directory properties to apply")
            return True

        serviceDirectoryURL = ags._url + "/system/handlers/rest/servicesdirectory"
        curProperties = ags._con.get(path=serviceDirectoryURL)
        updProperties = json.loads(json.dumps(curProperties))
        for k in servicesDirectory:
            if k == "enabled":
                if servicesDirectory[k] == True:
                    servicesDirectory[k] = "true"
                elif servicesDirectory[k] == False:
                    servicesDirectory[k] = "false"
            updProperties[k] = servicesDirectory[k]
        
        # Check that we are actually going to update something
        haveChanges = False
        for k in curProperties:
            if updProperties[k] != curProperties[k]:
                haveChanges = True
        for k in updProperties:
            if k not in curProperties:
                haveChanges = True

        if not haveChanges:
            logger.info("No Services Directory properties require changing")
            return True
        else:
            logger.info("Services Directory properties are changing from ")
            logger.info("current Properties: "+str(curProperties))
            logger.info("updated Properties: "+str(updProperties))

            url = ags._url + "/system/handlers/rest/servicesdirectory/edit"
            params = {
                "f" : "json",
                "allowedOrigins" : updProperties["allowedOrigins"],
                "arcgis.com.map" : updProperties["arcgis.com.map"],
                "arcgis.com.map.text" : updProperties["arcgis.com.map.text"],
                "jsapi.arcgis" : updProperties["jsapi.arcgis"],
                "jsapi.arcgis.css" : updProperties["jsapi.arcgis.css"],
                "jsapi.arcgis.sdk" : updProperties["jsapi.arcgis.sdk"],
                "servicesDirEnabled" : updProperties["enabled"]
            }
            logger.info("Updating services directory properties - this may take several minutes")
            result = ags._con.post(path=url, postdata=params)
            if "status" in result:
                return result["status"] == "success"
            else:
                return result
    except Exception as ex:
        logger.error("ERROR: Unexpected exception thrown", ex)
        return False
    
#Defining the JSON properties to security harden

logger.info('Defining General Properties')
generalProperties = {
    "access" : "private",
    "allSSL" : True,
    "culture" : "en-GB",
    "cultureFormat" : "gb",
    "units" : "metric"
}
logger.info(generalProperties)

logger.info('Defining Security Config')
securityConfig = {
    "defaultLevelForUser" : 2,
    "defaultRoleForUser" : "account_user",
    "enableAutomaticAccountCreation" : False,
    "allowedProxyHosts" : None,
    "disableServicesDirectory" : True
}
logger.info(securityConfig)

logger.info('Defining System Properties')
systemProperties = {
    "WebContextURL" : url,
    "privatePortalURL" : url,
    "disableSignup" : True
}
logger.info(systemProperties)

logger.info('Defining Server Services Directory Properties')
servicesDirectory = {
    "enabled" : False
}
logger.info(servicesDirectory)

#Instantiating the GIS and Server modules
logger.info('Logging into Portal and Server')
gis = GIS(url, username, password, verify_cert=False)
ags = server.Server(serverURL,gis=gis)
logger.info('Logged into Portal and Server succesfully')

#Calling functions and checking return status = True

# Set Portal General Properties

logger.info('Calling configureGeneralProperties function')
cgpBool = configureGeneralProperties(gis)
if cgpBool == True:
    logger.info('Successfully configured Portal general properties')
else:
    logger.error("ERROR: Couldn't configure Portal general properties")

# Set Portal Security Properties    
    
logger.info('Calling configureSecurity function')
csBool = configureSecurity(gis)
if csBool == True:
    logger.info('Successfully configured Portal security')
else:
    logger.error("ERROR: Couldn't configure Portal security")

# Set Portal System Properties   
    
logger.info('Calling configureSystemProperties function')
cspBool = configureSystemProperties(gis)
if cspBool == True:
    logger.info('Successfully configured Portal system properties')
else:
    logger.error("ERROR: Couldn't configure Portal system properties")

# Disable ArcGIS Server REST Directory

logger.info('Calling disableAGSServicesDirectory function')
dsdBool = disableAGSServicesDirectory(ags)
if dsdBool == True:
    logger.info('Successfully disabled ArcGIS Server services directory')
else:
    logger.error("ERROR: Failed to disable ArcGIS Server services directory")

logger.info('Finished Hardening security on the following ArcGIS Enterprise - '+url)